In [10]:
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

@flow.chunk
def user_input(trigger_data):
    print("Some initial data:", trigger_data)
    return input("Input something:")

@flow.chunk
def echo(trigger_data):
    print("You just input:", trigger_data)
    flow.data.set("trigger", trigger_data)

@flow.chunk
def triggered(trigger_data):
    print("I've been triggered")

flow.chunk(user_input).then(echo)
flow.on_data("trigger").then(triggered)

flow.start(user_input, "Let the show begin.")
pass

Some initial data: Let the show begin.
You just input: Agently TriggerFlow
I've been triggered


In [12]:
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

@flow.chunk
def count(_):
    counter = flow.data.get("counter", 0) + 1
    print("Times:", counter)
    flow.data.set("counter", counter)

@flow.chunk
def user_input(trigger_data):
    return input("Exit[Y/N]:")

(
    flow.wait(user_input)
        .if_condition(lambda trigger_data: trigger_data.lower() != "y")
            .then(count)
            .then(user_input)
)

flow.start(user_input)
pass

Times: 1
Times: 2


In [6]:
#!pip install Agently==3.4.1.2
import Agently
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

agent = (
    Agently.create_agent()
        .set_settings("current_model", "OAIClient")
        .set_settings("model.OAIClient.auth", { "api_key": "<Your-API-Key>" })
        .set_settings("model.OAIClient.url", "https://api.deepseek.com/v1")
        .set_settings("model.OAIClient.options", { "model": "deepseek-reasoner" })
)

@flow.chunk
def user_input(trigger_data):
    return input("[User]:")

@flow.chunk
def agent_response(trigger_data):
    response = (
        agent
            .input(trigger_data)
            .get_delta_generator()
    )
    for item in response:
        flow.result.put(item)
    flow.result.put_stop()

flow.chunk(user_input).to(agent_response)
response = flow.start()
print("[Assistant]: ", end="")
for delta in response:
    print(delta, end="", flush=True)

[Assistant]: Let \( D \) be the number of donkeys and \( S \) be the number of snails. We know the following:

1. The total number of heads is 20:
   \[
   D + S = 20
   \]
2. The total number of legs is 36. Donkeys have 4 legs each, and snails have 0 legs each:
   \[
   4D + 0S = 36
   \]

From the second equation, we can solve for \( D \):
   \[
   4D = 36 \implies D = 9
   \]

Substituting \( D = 9 \) into the first equation:
   \[
   9 + S = 20 \implies S = 11
   \]

Thus, there are \(\boxed{9}\) donkeys and \(\boxed{11}\) snails in the cage.

In [ ]:
import Agently
from agently_triggerflow import TriggerFlow

flow = TriggerFlow()

agent = (
    Agently.create_agent()
        .set_settings("current_model", "ERNIE")\
        .set_settings("model.ERNIE.auth", {
            "aistudio": "<Your-Baidu-AIStudio-Access-Token>",
        })
        .set_settings("model.ERNIE.options", { "model": "ernie-speed" })
)

@flow.chunk
def user_input(trigger_data):
    return input("[User]:")

@flow.chunk
def agent_response(trigger_data):
    response = (
        agent
            .input(trigger_data)
            .get_delta_generator()
    )
    reply = ""
    print("[Assistant]: ", end="")
    for delta in response:
        reply += delta
        print(delta, end="", flush=True)
    print("")

@flow.chunk
def add_chat_history(trigger_data):
    chat_history = flow.data.get("chat_history", [])
    chat_history.append({ "role": "user", "content": trigger_data["user_input"] })
    chat_history.append({ "role": "assistant", "content": trigger_data["user_input"] })
    flow.data.set("chat_history", chat_history)

(
    flow
        .wait(user_input)
        .if_condition(lambda trigger_data: trigger_data != "#exit")
            .to(agent_response)
        .else_condition()
            .to(flow.END)
)
(
    flow
        .wait_all(user_input, agent_response)
            .to(add_chat_history)
            .to(user_input)
)

flow.start()

[Assistant]: Good day to you too! Hope you're doing well. What are your plans for today? 请您告诉我今天有什么计划！
[Assistant]: My name is文心一言 (ERNIE Bot), a generative dialogue product based on the Wenxin Big Model technology developed by Baidu. I am designed to answer various questions and provide useful information to help people solve problems. I am also capable of performing tasks such as language understanding, generating responses, and even handling some basic AI conversational interactions.


'#exit'